In [ ]:
!pip install evaluate
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 775 kB/s 
     |████████████████████████████████| 451 kB 16.1 MB/s 
     |████████████████████████████████| 182 kB 74.2 MB/s 
     |████████████████████████████████| 212 kB 70.7 MB/s 
     |████████████████████████████████| 132 kB 83.0 MB/s 
     |████████████████████████████████| 127 kB 79.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 6.6 MB/s 
     |████████████████████████████████| 7.6 MB 51.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import re 
from tqdm import tqdm
import numpy as np
from nltk import tokenize
import nltk
import string
from keras.preprocessing.text import Tokenizer
import torch

*** INSTRUCTIONS ***  

__Upload/unload the files in the zip and check if the paths are appropriate
Just Run the entire file, Everything Works!__

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### Reading data 

In [ ]:
plato_full = open('/content/plato.txt',encoding="utf8").read()
shake_full = open('/content/shakespeare.txt',encoding="utf8").read()
print('Plato Length', len(plato_full), '\nShakespear Length', len(shake_full))

FileNotFoundError: ignored

In [ ]:
plato_sent = tokenize.sent_tokenize(plato_full)
shake_sent = tokenize.sent_tokenize(shake_full)

In [ ]:
len(plato_sent), len(shake_sent)

(13558, 101274)

In [ ]:
len(plato_full.split()), len(shake_full.split())

(255816, 961950)

In [ ]:
plato_sent[:2], shake_sent[:2]

(['\ufeffThe Project Gutenberg EBook of The Republic of Plato, by Plato\n\nThis eBook is for the use of anyone anywhere in the United States and most\nother parts of the world at no cost and with almost no restrictions\nwhatsoever.',
  'You may copy it, give it away or re-use it under the terms of\nthe Project Gutenberg License included with this eBook or online at\nwww.gutenberg.org.'],
 ['\ufeffThe Project Gutenberg eBook of The Complete Works of William Shakespeare, by William Shakespeare\n\nThis eBook is for the use of anyone anywhere in the United States and\nmost other parts of the world at no cost and with almost no restrictions\nwhatsoever.',
  'You may copy it, give it away or re-use it under the terms\nof the Project Gutenberg License included with this eBook or online at\nwww.gutenberg.org.'])

In [ ]:
def clean_data(sentences):
  sent_clean = []
  translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) 

  for data in sentences:
    data = data.replace('\n', ' ').replace('\r', ' ').replace('\ufeff', ' ')
    data = data.translate(translator)
    sent_clean.append(data.lower())
  return sent_clean

In [ ]:
plato_sent_clean = clean_data(plato_sent)
shake_sent_clean = clean_data(shake_sent)

In [ ]:
plato_sent_clean[:2], shake_sent_clean[:2]

([' the project gutenberg ebook of the republic of plato  by plato  this ebook is for the use of anyone anywhere in the united states and most other parts of the world at no cost and with almost no restrictions whatsoever ',
  'you may copy it  give it away or re use it under the terms of the project gutenberg license included with this ebook or online at www gutenberg org '],
 [' the project gutenberg ebook of the complete works of william shakespeare  by william shakespeare  this ebook is for the use of anyone anywhere in the united states and most other parts of the world at no cost and with almost no restrictions whatsoever ',
  'you may copy it  give it away or re use it under the terms of the project gutenberg license included with this ebook or online at www gutenberg org '])

### Dividing the sentences in train and test data and saving it in file

In [ ]:
l = []
plato_sent_new = []
for sent in plato_sent_clean:
  sent_len = len(sent.split())
  if sent_len > 5 and sent_len < 100:
    l.append(sent_len)
    plato_sent_new.append(' '.join(sent.split()))

In [ ]:
np.max(l), np.min(l), np.mean(l), plato_sent_new[np.argmin(l)], plato_sent_new[np.argmax(l)], len(plato_sent_new)

(99,
 6,
 24.97346547314578,
 'the introduction to the phaedrus vol',
 'and while weaker or coarser characters will extract good out of evil say in a corrupt state of the church or of society and live on happily allowing the evil to remain the finer or stronger natures may be crushed or spoiled by surrounding influences may become misanthrope and philanthrope by turns or in a few instances like the founders of the monastic orders or the reformers owing to some peculiarity in themselves or in their age may break away entirely from the world and from the church sometimes into great good sometimes into great evil sometimes into both',
 9384)

In [ ]:
l = []
shake_sent_new = []
for sent in shake_sent_clean:
  sent_len = len(sent.split())
  if sent_len > 5 and sent_len < 100:
    l.append(sent_len)
    shake_sent_new.append(' '.join(sent.split()))

In [ ]:
np.max(l), np.min(l), np.mean(l), shake_sent_new[np.argmin(l)], shake_sent_new[np.argmax(l)], len(shake_sent_new)

(99,
 6,
 17.631979166666667,
 'a room in the countess’s palace',
 'for do but note a wild and wanton herd or race of youthful and unhandled colts fetching mad bounds bellowing and neighing loud which is the hot condition of their blood if they but hear perchance a trumpet sound or any air of music touch their ears you shall perceive them make a mutual stand their savage eyes turn’d to a modest gaze by the sweet power of music therefore the poet did feign that orpheus drew trees stones and floods since naught so stockish hard and full of rage but music for the time doth change his nature',
 48000)

In [ ]:
len(shake_sent_new), len(plato_sent_new)

(48000, 9384)

In [ ]:
len(' '.join(shake_sent_new).split()), len(' '.join(plato_sent_new).split())

(846335, 234351)

In [ ]:
len(set(' '.join(shake_sent_new).split())), len(set(' '.join(plato_sent_new).split()))

(28981, 12853)

In [ ]:
shake_sent_temp = shake_sent_new[:1000]
plato_sent_temp = plato_sent_new[:1000]

In [ ]:
plato_train_sent = plato_sent_temp[0:int(0.8*len(plato_sent_temp))]
plato_test_sent = plato_sent_temp[int(0.8*len(plato_sent_temp)):]
shake_train_sent = shake_sent_temp[0:int(0.8*len(shake_sent_temp))]
shake_test_sent = shake_sent_temp[int(0.8*len(shake_sent_temp)):]

In [ ]:
plato_train_sent='.'.join(plato_train_sent)
plato_test_sent='.'.join(plato_test_sent)
shake_train_sent='.'.join(shake_train_sent)
shake_test_sent='.'.join(shake_test_sent)

In [ ]:
import pickle
with open("plato_train_sent.txt", "w") as fp:   #Pickling
    fp.write(plato_train_sent)
with open("plato_test_sent.txt", "w") as fp:   #Pickling
    fp.write(plato_test_sent)
with open("shake_train_sent.txt", "w") as fp:   #Pickling
    fp.write(shake_train_sent)
with open("shake_test_sent.txt", "w") as fp:   #Pickling
    fp.write(shake_test_sent)

### n-gram Language Model

In [ ]:
# Uncomment below block to generate the n_gram files for training and testing
# prepare data

# shake_train_ngram = open('/content/shake_train_full.txt', 'r').read()
# shake_test_ngram = open('/content/shake_test_full.txt', 'r').read()

In [ ]:
# shake_train_ngram = ['<s> '+line+ ' </s>'for line in shake_train_ngram.split('\n')]
# shake_test_ngram = ['<s> '+line+ ' </s>'for line in shake_test_ngram.split('\n')]
# with open("shake_train_ngram.txt", "w") as fp:   #Pickling
#     fp.write('\n'.join(shake_train_ngram))
# with open("shake_test_ngram.txt", "w") as fp:   #Pickling
#     fp.write('\n'.join(shake_test_ngram))

In [ ]:
import sys
import numpy as np
import math
import random

class LanguageModel:
    UNK = "<UNK>"
    SENT_BEGIN = "<s>"
    SENT_END = "</s>"


    def __init__(self, n_gram, is_laplace_smoothing):
        """Initializes an untrained LanguageModel
        Parameters:
          n_gram (int): the n-gram order of the language model to create
          is_laplace_smoothing (bool): whether or not to use Laplace smoothing
        """
        self.n_gram = int(n_gram)
        self.is_laplace_smoothing = bool(is_laplace_smoothing)
        self.training_file_path = ''

    def train(self, training_file_path):
        """Trains the language model on the given data. Assumes that the given data
        has tokens that are white-space separated, has one sentence per line, and
        that the sentences begin with <s> and end with </s>
        Parameters:
          training_file_path (str): the location of the training data to read

        Returns:
        None
        """
        # storing path to display later
        self.training_file_path = str (training_file_path)

        # Gather contents from the file
        f = open(training_file_path, "r")
        files = f.read()
        f.close()
        self.tokens = files.split()

        # storing frequency of unique tokens in the tokens
        self.uniq_tokens = {}
        for tok in self.tokens:
            if tok not in self.uniq_tokens:
                self.uniq_tokens[tok] = 1
            else:
                self.uniq_tokens[tok] += 1

        # replacing words with frequency 1 with <UNK>
        self.tokens_unk = self.tokens.copy()
        for i in self.uniq_tokens:
            if self.uniq_tokens[i] == 1:
                for k, j in enumerate(self.tokens):
                    if i == j:
                        self.tokens_unk[k] = "<UNK>"

        # Creating a list of all possible n-grams from the tokens_unk and storing in ngrams_lst list
        ngrams = zip(*[self.tokens_unk[i:] for i in range(self.n_gram)])
        # print('gs', [i for i in ngrams])
        ngrams_lst = [" ".join(ngs) for ngs in ngrams]
        # ngram_list stores the ngrams formed

        # ngrams_dict(dict) has the frequencies of all unique ngrams in the tokens_unk list.
        self.ngrams_dict = {}
        for ngram in ngrams_lst:
            if ngram not in self.ngrams_dict:
                self.ngrams_dict[ngram] = 1
            else:
                self.ngrams_dict[ngram] += 1

        # Creating dictionary for n-1 gram and generating the word frequencies
        self.n_minus_one = {}
        if self.n_gram == 1:
            self.n_minus_one = self.ngrams_dict.copy()
        else:
            nminus = zip(*[self.tokens_unk[i:] for i in range(self.n_gram - 1)])
            self.nminus_lst = [" ".join(gram) for gram in nminus]
            for gram in self.nminus_lst:
                if gram not in self.n_minus_one:
                    self.n_minus_one[gram] = 1
                else:
                    self.n_minus_one[gram] += 1

    def score(self, sentence):
        """Calculates the probability score for a given string representing a single sentence.
        Parameters:
          sentence (str): a sentence with tokens separated by whitespace to calculate the score of

        Returns:
          float: the probability value of the given string for this model
        """
        # replacing the tokens of frequency 1 with "<UNK>"
        tokens_list = sentence.split()
        tokens_lst_new = tokens_list.copy()
        for tok in tokens_list:
            if tok not in self.uniq_tokens or self.uniq_tokens[tok] == 1:
                for idx, val in enumerate(tokens_list):
                    if tok == val:
                        tokens_lst_new[idx] = "<UNK>"

        # generating ngrams list
        score_ngram_lst = zip(*[tokens_lst_new[i:] for i in range(self.n_gram)])
        score_ngram_lst = [" ".join(ngram) for ngram in score_ngram_lst]


        # generating n-1 grams list
        sentence_n_minus_grams = []
        if self.n_gram != 1:
            score_ngram = zip(*[tokens_lst_new[i:] for i in range(self.n_gram-1)])
            sentence_n_minus_grams = [" ".join(gram) for gram in score_ngram]
        elif self.n_gram == 1:
            sentence_n_minus_grams = score_ngram_lst

        # calculating probability - laplace and non-laplace smoothing
        score = 1
        if self.is_laplace_smoothing == False:
            for i in range(len(score_ngram_lst)):
                if score_ngram_lst[i] not in self.ngrams_dict:
                    return 0
                else:
                    if self.n_gram == 1:
                        score = (score * self.ngrams_dict[score_ngram_lst[i]]) / len(self.tokens)
                    else:
                        score = (score * self.ngrams_dict[score_ngram_lst[i]]) / self.n_minus_one[sentence_n_minus_grams[i]]
        else:
            for i in range(len(score_ngram_lst)):
                if score_ngram_lst[i] not in self.ngrams_dict:
                    score = (score/(self.n_minus_one[sentence_n_minus_grams[i]]+len(set(self.tokens_unk))))
                else:
                    if self.n_gram == 1:
                        score = (score*(self.ngrams_dict[score_ngram_lst[i]]+1)) / (len(self.tokens_unk)+len(set(self.tokens_unk)))
                    else:
                        score = (score*(self.ngrams_dict[score_ngram_lst[i]]+1)) / (self.n_minus_one[sentence_n_minus_grams[i]]+len(set(self.tokens_unk)))
        return score

    def perplexity(self, test_sequence):
        """
        Measures the perplexity for the given test sequence with this trained model.
                 As described in the text, you may assume that this sequence
                 may consist of many sentences "glued together".
        Parameters:
          test_sequence (string): a sequence of space-separated tokens to measure the perplexity of
        Returns:
          float: the perplexity of the given sequence
        """
        score = self.score(test_sequence)
        len_testseq = len(test_sequence.split())

        # checking if the score is equal to zero to avoid divide by infinity error
        if score != 0:
            base = math.exp(math.log(score)/len_testseq)  # applying formula for perplexity
            return 1/base
        else:
            return float('inf') # else returning infinity

In [ ]:
bigram_plato = LanguageModel(2, True)
bigram_plato.train("plato_train_ngram.txt")

In [ ]:
bigram_plato_test = open('/content/plato_test_ngram.txt', 'r').read()
bigram_plato_test = bigram_plato_test.split('\n')
bigram_perplexity = []
for line in bigram_plato_test:
  bigram_perplexity.append(bigram_plato.perplexity(line))

In [ ]:
print('The perplexity for bigram model for plato data is:', np.mean(bigram_perplexity))

The perplexity for bigram model for plato data is: 348.4495100071398


In [ ]:
print('The perplexity for bigram model for plato data is (No Smoothing):', np.mean(bigram_perplexity))

The perplexity for bigram model for plato data is (No Smoothing): 348.4495100071398


In [ ]:
trigram_plato = LanguageModel(3, True)
trigram_plato.train("plato_train_ngram.txt")

In [ ]:
trigram_plato_test = open('/content/plato_test_ngram.txt', 'r').read()
trigram_plato_test = trigram_plato_test.split('\n')
trigram_perplexity = []
for line in trigram_plato_test:
  trigram_perplexity.append(trigram_plato.perplexity(line))

In [ ]:
print('The perplexity for Trigram model for plato data is:', np.mean(trigram_perplexity))

The perplexity for Trigram model for plato data is: 481.35901397061485


In [ ]:
pentagram_plato = LanguageModel(5, True)
pentagram_plato.train("plato_train_ngram.txt")

In [ ]:
pentagram_plato_test = open('/content/plato_test_ngram.txt', 'r').read()
pentagram_plato_test = pentagram_plato_test.split('\n')
pentagram_perplexity = []
for line in pentagram_plato_test:
  pentagram_perplexity.append(pentagram_plato.perplexity(line))

In [ ]:
print('The perplexity for Pentagram model for plato data is:', np.mean(pentagram_perplexity))

The perplexity for Pentagram model for plato data is: 295.34152214208916


In [ ]:
bigram_shake = LanguageModel(2, True)
bigram_shake.train("shake_train_ngram.txt")

In [ ]:
bigram_shake_test = open('shake_test_ngram.txt', 'r').read()
bigram_shake_test = bigram_shake_test.split('\n')
bigram_shake_perplexity = []
# testing for first 2000 rows
for line in bigram_shake_test[:2000]:
  bigram_shake_perplexity.append(bigram_shake.perplexity(line))

In [ ]:
bigram_shake_perplexity = np.array(bigram_shake_perplexity)
bigram_shake_perplexity = bigram_shake_perplexity[bigram_shake_perplexity != np.inf]

In [ ]:
print('The perplexity for bigram model for Shakespeare data is:', np.mean(bigram_shake_perplexity))

The perplexity for bigram model for Shakespeare data is: 1200.6132538457935


In [ ]:
trigram_shake = LanguageModel(3, True)
trigram_shake.train("shake_train_ngram.txt")

In [ ]:
trigram_shake_test = open('shake_train_ngram.txt', 'r').read()
trigram_shake_test = trigram_shake_test.split('\n')
trigram_shake_perplexity = []
for line in trigram_shake_test[:2000]:
  trigram_shake_perplexity.append(trigram_shake.perplexity(line))

In [ ]:
trigram_shake_perplexity = np.array(trigram_shake_perplexity)
trigram_shake_perplexity = trigram_shake_perplexity[trigram_shake_perplexity != np.inf]
print('The perplexity for trigram model for Shakespeare data is:', np.mean(trigram_shake_perplexity))

The perplexity for trigram model for Shakespeare data is: 2113.267431506607


In [ ]:
pentagram_shake = LanguageModel(5, True)
pentagram_shake.train("shake_train_ngram.txt")

In [ ]:
pentagram_shake_test = open('shake_train_ngram.txt', 'r').read()
pentagram_shake_test = pentagram_shake_test.split('\n')
pentagram_shake_perplexity = []
for line in pentagram_shake_test[:2000]:
  pentagram_shake_perplexity.append(pentagram_shake.perplexity(line))

In [ ]:
pentagram_shake_perplexity = np.array(pentagram_shake_perplexity)
pentagram_shake_perplexity = pentagram_shake_perplexity[pentagram_shake_perplexity != np.inf]
print('The perplexity for pentagram model for Shakespeare data is:', np.mean(pentagram_shake_perplexity))

The perplexity for pentagram model for Shakespeare data is: 1316.6049909102373


### Modeling using GPT

In [ ]:
from transformers import GPT2TokenizerFast, GPT2LMHeadModel, TrainingArguments, Trainer, pipeline, DataCollatorForLanguageModeling
import evaluate
import datasets
import logging
logging.basicConfig(level='ERROR')

In [ ]:
class gpt_modelling():

  def __init__(self, path, max_seq_length = 100):
    self.dataset = datasets.load_dataset(path)
    self.tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
    self.tokenizer.pad_token = self.tokenizer.eos_token
    self.model = GPT2LMHeadModel.from_pretrained('gpt2')
    self.max_seq_length = max_seq_length
    self.data_collator = DataCollatorForLanguageModeling(tokenizer=self.tokenizer, mlm=False)
    self.model_name = ''

    self.tokenized_data = self.generate_data(self.dataset)

  def tokenize_function(self, examples):
    # Remove empty lines
    examples["text"] = [line for line in examples["text"] if len(line) > 0 and not line.isspace()]
    return self.tokenizer(
        examples["text"],
        truncation=True,
        max_length=self.max_seq_length,)

  def generate_data(self, data):
    return data.map(
    self.tokenize_function,
    batched=True,
    remove_columns=["text"])
  
  def get_vocab_details(self):
    print('vocabulary size: %d, max sequence length: %d' % (self.tokenizer.vocab_size, self.tokenizer.model_max_length))
    print(self.tokenizer.pretrained_vocab_files_map)

  def calculate_perplexity(self, test_set_len=100, data_type='test'):
      if self.model_name == '':
        self.model_name = 'gpt2'
      # # Given a model and an input text sequence, perplexity measures how likely the model is to generate the input text sequence.
      if test_set_len > len(self.dataset[data_type]['text']):
        test_set_len = len(self.dataset[data_type]['text'])
      perplexity = evaluate.load("perplexity", module_type="metric")
      input_texts = []
      for d in self.dataset[data_type]['text'][:test_set_len]:
        if len(d) > self.max_seq_length :
          d = d[:self.max_seq_length]
        input_texts.append(d)
      results = perplexity.compute(model_id=self.model_name,
                                  add_start_token=False,
                                  predictions=input_texts)
      # results.keys = ['perplexities', 'mean_perplexity']
      return results['mean_perplexity']
    
  def evaluate(self):
    print(self.trainer.evaluate(self.tokenized_data['test']))
  
  def train(self, batch_size=64, lr=0.0005, epochs=3, model_name='gpt_model'):
    self.model_name = model_name
    training_args = TrainingArguments(
        output_dir = model_name, 
        overwrite_output_dir = True, 
        per_device_train_batch_size = batch_size, 
        per_device_eval_batch_size = batch_size, 
        learning_rate = lr, 
        num_train_epochs = epochs)
    
    self.trainer = Trainer(
        model = self.model,
        args = training_args,
        data_collator=self.data_collator,
        train_dataset = self.tokenized_data['train'],
        eval_dataset = self.tokenized_data['test'])
    self.trainer.train()
    self.trainer.save_model()
    self.tokenizer.save_pretrained(self.model_name)
    return self.trainer

    def next_word_generator(self, text=''):
      generator = pipeline('text-generation', tokenizer=self.tokenizer, model=self.model) 
      while(True):
        text = input('Enter the text: ')
        if text == 'exit':
          break
        length= len(tokenizer.encode(text, return_tensors='pt')[0])
        
        max_length = 1

        print('Next Word: ')
        print(generator(text , max_new_tokens=max_length)[0]['generated_text'].split(' ')[-1])
        print(generator(text , max_new_tokens=max_length , num_beams = 5)[0]['generated_text'].split(' ')[-1])
        print(generator(text , max_new_tokens=max_length , do_sample=True,temperature = 0.7)[0]['generated_text'].split(' ')[-1])   

In [ ]:
# group the train and test data of plato under one folder and mention it as the path
plato_gpt = gpt_modelling('/content/plato_data', 35)

  0%|          | 0/2 [00:00<?, ?it/s]

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/f27b190eeac4c2302d24068eabf5e9d6044389ae/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/f27b190eeac4c2302d24068eabf5e9d6044389ae/merges.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/f27b190eeac4c2302d24068eabf5e9d6044389ae/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/f27b190eeac4c2302d24068eabf5e9d6044389ae/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_ran

In [ ]:
plato_gpt.get_vocab_details()

vocabulary size: 50257, max sequence length: 1024
{'vocab_file': {'gpt2': 'https://huggingface.co/gpt2/resolve/main/vocab.json', 'gpt2-medium': 'https://huggingface.co/gpt2-medium/resolve/main/vocab.json', 'gpt2-large': 'https://huggingface.co/gpt2-large/resolve/main/vocab.json', 'gpt2-xl': 'https://huggingface.co/gpt2-xl/resolve/main/vocab.json', 'distilgpt2': 'https://huggingface.co/distilgpt2/resolve/main/vocab.json'}, 'merges_file': {'gpt2': 'https://huggingface.co/gpt2/resolve/main/merges.txt', 'gpt2-medium': 'https://huggingface.co/gpt2-medium/resolve/main/merges.txt', 'gpt2-large': 'https://huggingface.co/gpt2-large/resolve/main/merges.txt', 'gpt2-xl': 'https://huggingface.co/gpt2-xl/resolve/main/merges.txt', 'distilgpt2': 'https://huggingface.co/distilgpt2/resolve/main/merges.txt'}, 'tokenizer_file': {'gpt2': 'https://huggingface.co/gpt2/resolve/main/tokenizer.json', 'gpt2-medium': 'https://huggingface.co/gpt2-medium/resolve/main/tokenizer.json', 'gpt2-large': 'https://huggingf

In [ ]:
plato_train = plato_gpt.train(batch_size=8, epochs=10, model_name='gpt_plato')

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 2574
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3220
  Number of trainable parameters = 124439808
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,3.377900
1000,1.992200
1500,1.046600
2000,0.740300
2500,0.575700
3000,0.476200


Saving model checkpoint to gpt_plato/checkpoint-500
Configuration saved in gpt_plato/checkpoint-500/config.json
Model weights saved in gpt_plato/checkpoint-500/pytorch_model.bin
Saving model checkpoint to gpt_plato/checkpoint-1000
Configuration saved in gpt_plato/checkpoint-1000/config.json
Model weights saved in gpt_plato/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to gpt_plato/checkpoint-1500
Configuration saved in gpt_plato/checkpoint-1500/config.json
Model weights saved in gpt_plato/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to gpt_plato/checkpoint-2000
Configuration saved in gpt_plato/checkpoint-2000/config.json
Model weights saved in gpt_plato/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to gpt_plato/checkpoint-2500
Configuration saved in gpt_plato/checkpoint-2500/config.json
Model weights saved in gpt_plato/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to gpt_plato/checkpoint-3000
Configuration saved in gpt_plato/checkpoint-3

In [ ]:
# test perplexity value before fine tuning
plato_gpt.calculate_perplexity(data_type='test')

loading configuration file gpt_plato/config.json
Model config GPT2Config {
  "_name_or_path": "gpt_plato",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "use_cache": true,
  "vocab_size": 50257
}

loading weight

  0%|          | 0/7 [00:00<?, ?it/s]

67.03158487319946

In [ ]:
# train perplexity value before fine tuning
plato_gpt.calculate_perplexity(data_type='train')

loading configuration file gpt_plato/config.json
Model config GPT2Config {
  "_name_or_path": "gpt_plato",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "use_cache": true,
  "vocab_size": 50257
}

loading weight

  0%|          | 0/7 [00:00<?, ?it/s]

67.03158487319946

In [ ]:
# group the train and test data of shakespeare under one folder and mention it as the path
shake_gpt = gpt_modelling('/content/shake_data', 25)

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/shake_data-f87e520bf46297b3/0.0.0/99cc88223027054f94ce0c7fd69d10eb172910fa0615671283a3c8e5e7af2f9c. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/f27b190eeac4c2302d24068eabf5e9d6044389ae/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/f27b190eeac4c2302d24068eabf5e9d6044389ae/merges.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/f27b190eeac4c2302d24068eabf5e9d6044389ae/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/f27b190eeac4c2302d24068eabf5e9d6044389ae/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_ran

  0%|          | 0/40 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

In [ ]:
shake_train = shake_gpt.train(batch_size=8, epochs=10, lr=0.0001, model_name='gpt_shake')

In [ ]:
# The test preplexity value before fine tuning
shake_gpt.calculate_perplexity(data_type='test')

In [ ]:
# The train preplexity value before fine tuning
shake_gpt.calculate_perplexity(data_type='train')

Using pad_token, but it is not set yet.


  0%|          | 0/7 [00:00<?, ?it/s]

15042.61944530487